For PCA reduction(partial parameter tuning)

In [32]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

# Load the dataset
def load_data(file_path):
    df = pd.read_excel(file_path)
    keywords = df[[f'Keywords_Vector_{i}' for i in range(20)]]
    custom_data_structures = df[[f'Custom_Data_Structures_Vector_{i}' for i in range(20)]]
    final_marks = df["Final_Marks"]
    return keywords, custom_data_structures, final_marks

# Function to pad vectors with zeros to match length
def pad_with_zeros(vector, max_length):
    vector = np.atleast_2d(vector)
    if vector.shape[1] < max_length:
        padding = np.zeros((vector.shape[0], max_length - vector.shape[1]))
        vector = np.hstack((vector, padding))
    return vector

# Reduction and combination strategies
def combine_vectors(keywords, custom_data_structures, weights, fusion):
    # Apply weights
    weighted_keywords = weights[0] * keywords
    weighted_custom_data_structures = weights[1] * custom_data_structures

    # Apply fusion
    if fusion == "add":
        max_length = max(weighted_keywords.shape[1], weighted_custom_data_structures.shape[1])
        weighted_keywords = pad_with_zeros(weighted_keywords, max_length)
        weighted_custom_data_structures = pad_with_zeros(weighted_custom_data_structures, max_length)
        combined_vectors = weighted_keywords + weighted_custom_data_structures
    elif fusion == "concatenate":
        combined_vectors = np.hstack((weighted_keywords, weighted_custom_data_structures))
    return combined_vectors

# Model definitions
catboost_params = {
    'verbose': 0,
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 5,
    'early_stopping_rounds': 15,'random_state': 43
}

xgboost_params = {
    'n_estimators': 500,
    'learning_rate': 0.01,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1,'random_state': 43
}

random_forest_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'min_samples_split': 5,
    'min_samples_leaf': 3,'random_state': 43
}

bagging_params = {
    'n_estimators': 100,
    'max_samples': 0.8,
    'max_features': 1.0,'random_state': 43
}

# Base models
models = {
    "CatBoost Regressor": CatBoostRegressor(**catboost_params),
    "XGBoost Regressor": XGBRegressor(**xgboost_params),
    "Random Forest Regressor": RandomForestRegressor(**random_forest_params),
    "Bagging Regressor": BaggingRegressor(**bagging_params)
}

# Create a stacking regressor using the models as base learners
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=CatBoostRegressor(n_estimators=200, learning_rate=0.05, max_depth=3,verbose=0),
)

# Add the stacking model to the models dictionary
models["Stacking Regressor"] = stacking_model

# Load data without reduction
file_path = "/content/pca.xlsx"
keywords, custom_data_structures, final_marks = load_data(file_path)

# Weights and fusion types
weights_options = [(1, 1)]
fusions = ["add"]

# Print results directly
for weights in weights_options:
    for fusion in fusions:
        combined_vectors = combine_vectors(keywords, custom_data_structures, weights, fusion)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(combined_vectors, final_marks, test_size=0.2, random_state=42, stratify=final_marks)

        print(f"\nFusion method: {fusion}")
        for name, model in models.items():
            model.fit(X_train, y_train)

            # Predict and calculate metrics for training set
            y_train_pred = np.round(model.predict(X_train))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            train_mse = mean_squared_error(y_train, y_train_pred)
            train_rmse = np.sqrt(train_mse)
            train_r2 = r2_score(y_train, y_train_pred)
            train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

            # Predict and calculate metrics for testing set
            y_test_pred = np.round(model.predict(X_test))
            test_mae = mean_absolute_error(y_test, y_test_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)
            test_rmse = np.sqrt(test_mse)
            test_r2 = r2_score(y_test, y_test_pred)
            test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

            # Print results
            print(f"\nModel: {name}")
            print("Train Metrics:")
            print(f"MAE: {train_mae}, MSE: {train_mse}, RMSE: {train_rmse}, R2: {train_r2}, MAPE: {train_mape}")
            print("Test Metrics:")
            print(f"MAE: {test_mae}, MSE: {test_mse}, RMSE: {test_rmse}, R2: {test_r2}, MAPE: {test_mape}")



Fusion method: add

Model: CatBoost Regressor
Train Metrics:
MAE: 0.5431472081218274, MSE: 0.7766497461928934, RMSE: 0.8812773378414388, R2: 0.8679843037055672, MAPE: 9144364725625.494
Test Metrics:
MAE: 1.3238866396761133, MSE: 3.064777327935223, RMSE: 1.7506505442078446, R2: 0.4755706264128928, MAPE: 0.3558399203136045

Model: XGBoost Regressor
Train Metrics:
MAE: 0.9604060913705583, MSE: 1.6385786802030458, RMSE: 1.280069795051444, R2: 0.7214727401733398, MAPE: 18288729451250.977
Test Metrics:
MAE: 1.4696356275303644, MSE: 3.445344129554656, RMSE: 1.8561638207751643, R2: 0.41044992208480835, MAPE: 0.40919606709080397

Model: Random Forest Regressor
Train Metrics:
MAE: 1.3147208121827412, MSE: 2.912690355329949, RMSE: 1.7066605858605715, R2: 0.5048979965114674, MAPE: 22860911814063.777
Test Metrics:
MAE: 1.6842105263157894, MSE: 4.477732793522267, RMSE: 2.116065403885775, R2: 0.23379275140377742, MAPE: 0.4909163935479725

Model: Bagging Regressor
Train Metrics:
MAE: 0.72994923857868

In [33]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

# Load the dataset
def load_data(file_path):
    df = pd.read_excel(file_path)
    keywords = df[[f'Keywords_Vector_{i}' for i in range(20)]]
    custom_data_structures = df[[f'Custom_Data_Structures_Vector_{i}' for i in range(20)]]
    final_marks = df["Final_Marks"]
    return keywords, custom_data_structures, final_marks

# Function to pad vectors with zeros to match length
def pad_with_zeros(vector, max_length):
    vector = np.atleast_2d(vector)
    if vector.shape[1] < max_length:
        padding = np.zeros((vector.shape[0], max_length - vector.shape[1]))
        vector = np.hstack((vector, padding))
    return vector

# Reduction and combination strategies
def combine_vectors(keywords, custom_data_structures, weights, fusion):
    # Apply weights
    weighted_keywords = weights[0] * keywords
    weighted_custom_data_structures = weights[1] * custom_data_structures

    # Apply fusion
    if fusion == "add":
        max_length = max(weighted_keywords.shape[1], weighted_custom_data_structures.shape[1])
        weighted_keywords = pad_with_zeros(weighted_keywords, max_length)
        weighted_custom_data_structures = pad_with_zeros(weighted_custom_data_structures, max_length)
        combined_vectors = weighted_keywords + weighted_custom_data_structures
    elif fusion == "concatenate":
        combined_vectors = np.hstack((weighted_keywords, weighted_custom_data_structures))
    return combined_vectors

# Model definitions
catboost_params = {
    'verbose': 0,
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 5,
    'early_stopping_rounds': 15,'random_state': 43
}

xgboost_params = {
    'n_estimators': 500,
    'learning_rate': 0.01,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1,'random_state': 43
}

random_forest_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'min_samples_split': 5,
    'min_samples_leaf': 3,'random_state': 43
}

bagging_params = {
    'n_estimators': 100,
    'max_samples': 0.8,
    'max_features': 1.0,'random_state': 43
}

# Base models
models = {
    "CatBoost Regressor": CatBoostRegressor(**catboost_params),
    "XGBoost Regressor": XGBRegressor(**xgboost_params),
    "Random Forest Regressor": RandomForestRegressor(**random_forest_params),
    "Bagging Regressor": BaggingRegressor(**bagging_params)
}

# Create a stacking regressor using the models as base learners
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=CatBoostRegressor(n_estimators=200, learning_rate=0.05, max_depth=3,verbose=0),
)

# Add the stacking model to the models dictionary
models["Stacking Regressor"] = stacking_model

# Load data without reduction
file_path = "/content/pca.xlsx"
keywords, custom_data_structures, final_marks = load_data(file_path)

# Weights and fusion types
weights_options = [(1, 1)]
fusions = ["concatenate"]

# Print results directly
for weights in weights_options:
    for fusion in fusions:
        combined_vectors = combine_vectors(keywords, custom_data_structures, weights, fusion)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(combined_vectors, final_marks, test_size=0.2, random_state=42, stratify=final_marks)

        print(f"\nFusion method: {fusion}")
        for name, model in models.items():
            model.fit(X_train, y_train)

            # Predict and calculate metrics for training set
            y_train_pred = np.round(model.predict(X_train))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            train_mse = mean_squared_error(y_train, y_train_pred)
            train_rmse = np.sqrt(train_mse)
            train_r2 = r2_score(y_train, y_train_pred)
            train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

            # Predict and calculate metrics for testing set
            y_test_pred = np.round(model.predict(X_test))
            test_mae = mean_absolute_error(y_test, y_test_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)
            test_rmse = np.sqrt(test_mse)
            test_r2 = r2_score(y_test, y_test_pred)
            test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

            # Print results
            print(f"\nModel: {name}")
            print("Train Metrics:")
            print(f"MAE: {train_mae}, MSE: {train_mse}, RMSE: {train_rmse}, R2: {train_r2}, MAPE: {train_mape}")
            print("Test Metrics:")
            print(f"MAE: {test_mae}, MSE: {test_mse}, RMSE: {test_rmse}, R2: {test_r2}, MAPE: {test_mape}")



Fusion method: concatenate

Model: CatBoost Regressor
Train Metrics:
MAE: 0.584771573604061, MSE: 0.8243654822335026, RMSE: 0.9079457485078625, R2: 0.8598735354364976, MAPE: 9144364725625.5
Test Metrics:
MAE: 1.291497975708502, MSE: 2.7732793522267207, RMSE: 1.6653165921910227, R2: 0.5254503026325384, MAPE: 0.336053274211169

Model: XGBoost Regressor
Train Metrics:
MAE: 0.9187817258883249, MSE: 1.5401015228426396, RMSE: 1.241008268643944, R2: 0.738211989402771, MAPE: 18288729451250.97
Test Metrics:
MAE: 1.408906882591093, MSE: 3.2550607287449393, RMSE: 1.8041786853704207, R2: 0.44301027059555054, MAPE: 0.37971370734528626

Model: Random Forest Regressor
Train Metrics:
MAE: 1.2873096446700507, MSE: 2.7593908629441626, RMSE: 1.66114143375697, R2: 0.5309559966950743, MAPE: 22860911814063.758
Test Metrics:
MAE: 1.5870445344129556, MSE: 4.089068825910931, RMSE: 2.0221446105338092, R2: 0.3002989863633049, MAPE: 0.4370525673157253

Model: Bagging Regressor
Train Metrics:
MAE: 0.6974619289340

For Information Gain(No parameter tuning)

In [36]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

# Load the dataset
def load_data(file_path):
    df = pd.read_excel(file_path)
    keywords = df[[f'Keywords_Vector_{i}' for i in range(20)]]
    custom_data_structures = df[[f'Custom_Data_Structures_Vector_{i}' for i in range(20)]]
    final_marks = df["Final_Marks"]
    return keywords, custom_data_structures, final_marks

# Function to pad vectors with zeros to match length
def pad_with_zeros(vector, max_length):
    vector = np.atleast_2d(vector)
    if vector.shape[1] < max_length:
        padding = np.zeros((vector.shape[0], max_length - vector.shape[1]))
        vector = np.hstack((vector, padding))
    return vector

# Reduction and combination strategies
def combine_vectors(keywords, custom_data_structures, weights, fusion):
    # Apply weights
    weighted_keywords = weights[0] * keywords
    weighted_custom_data_structures = weights[1] * custom_data_structures

    # Apply fusion
    if fusion == "add":
        max_length = max(weighted_keywords.shape[1], weighted_custom_data_structures.shape[1])
        weighted_keywords = pad_with_zeros(weighted_keywords, max_length)
        weighted_custom_data_structures = pad_with_zeros(weighted_custom_data_structures, max_length)
        combined_vectors = weighted_keywords + weighted_custom_data_structures
    elif fusion == "concatenate":
        combined_vectors = np.hstack((weighted_keywords, weighted_custom_data_structures))
    return combined_vectors

# Model definitions
catboost_params = {
    'verbose': 0,
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 5,
    'early_stopping_rounds': 15,'random_state': 43
}

xgboost_params = {
    'n_estimators': 500,
    'learning_rate': 0.01,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1,'random_state': 43
}

random_forest_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'min_samples_split': 5,
    'min_samples_leaf': 3,'random_state': 43
}

bagging_params = {
    'n_estimators': 100,
    'max_samples': 0.8,
    'max_features': 1.0,'random_state': 43
}

# Base models
models = {
    "CatBoost Regressor": CatBoostRegressor(**catboost_params),
    "XGBoost Regressor": XGBRegressor(**xgboost_params),
    "Random Forest Regressor": RandomForestRegressor(**random_forest_params),
    "Bagging Regressor": BaggingRegressor(**bagging_params)
}

# Create a stacking regressor using the models as base learners
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=CatBoostRegressor(n_estimators=200, learning_rate=0.05, max_depth=3,verbose=0),
)

# Add the stacking model to the models dictionary
models["Stacking Regressor"] = stacking_model

# Load data without reduction
file_path = "/content/IG.xlsx"
keywords, custom_data_structures, final_marks = load_data(file_path)

# Weights and fusion types
weights_options = [(1, 1)]
fusions = ["add"]

# Print results directly
for weights in weights_options:
    for fusion in fusions:
        combined_vectors = combine_vectors(keywords, custom_data_structures, weights, fusion)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(combined_vectors, final_marks, test_size=0.2, random_state=42, stratify=final_marks)

        print(f"\nFusion method: {fusion}")
        for name, model in models.items():
            model.fit(X_train, y_train)

            # Predict and calculate metrics for training set
            y_train_pred = np.round(model.predict(X_train))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            train_mse = mean_squared_error(y_train, y_train_pred)
            train_rmse = np.sqrt(train_mse)
            train_r2 = r2_score(y_train, y_train_pred)
            train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

            # Predict and calculate metrics for testing set
            y_test_pred = np.round(model.predict(X_test))
            test_mae = mean_absolute_error(y_test, y_test_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)
            test_rmse = np.sqrt(test_mse)
            test_r2 = r2_score(y_test, y_test_pred)
            test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

            # Print results
            print(f"\nModel: {name}")
            print("Train Metrics:")
            print(f"MAE: {train_mae}, MSE: {train_mse}, RMSE: {train_rmse}, R2: {train_r2}, MAPE: {train_mape}")
            print("Test Metrics:")
            print(f"MAE: {test_mae}, MSE: {test_mse}, RMSE: {test_rmse}, R2: {test_r2}, MAPE: {test_mape}")



Fusion method: add

Model: CatBoost Regressor
Train Metrics:
MAE: 0.5299492385786801, MSE: 0.7573604060913706, RMSE: 0.8702645609763565, R2: 0.8712631249207231, MAPE: 9144364725625.486
Test Metrics:
MAE: 1.3562753036437247, MSE: 3.2753036437246963, RMSE: 1.8097799987083227, R2: 0.43954641580981546, MAPE: 0.38681961313540264

Model: XGBoost Regressor
Train Metrics:
MAE: 0.9898477157360406, MSE: 1.700507614213198, RMSE: 1.3040351276760906, R2: 0.7109460234642029, MAPE: 18288729451250.992
Test Metrics:
MAE: 1.4736842105263157, MSE: 3.611336032388664, RMSE: 1.900351554946785, R2: 0.3820462226867676, MAPE: 0.4324481074481074

Model: Random Forest Regressor
Train Metrics:
MAE: 1.3766497461928935, MSE: 3.0639593908629443, RMSE: 1.750416919154675, R2: 0.47918513540314, MAPE: 27433094176876.496
Test Metrics:
MAE: 1.6518218623481782, MSE: 4.42914979757085, RMSE: 2.104554536611216, R2: 0.24210603077371828, MAPE: 0.507211940106677

Model: Bagging Regressor
Train Metrics:
MAE: 0.7401015228426396, 

In [37]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression

# Load the dataset
def load_data(file_path):
    df = pd.read_excel(file_path)
    keywords = df[[f'Keywords_Vector_{i}' for i in range(20)]]
    custom_data_structures = df[[f'Custom_Data_Structures_Vector_{i}' for i in range(20)]]
    final_marks = df["Final_Marks"]
    return keywords, custom_data_structures, final_marks

# Function to pad vectors with zeros to match length
def pad_with_zeros(vector, max_length):
    vector = np.atleast_2d(vector)
    if vector.shape[1] < max_length:
        padding = np.zeros((vector.shape[0], max_length - vector.shape[1]))
        vector = np.hstack((vector, padding))
    return vector

# Reduction and combination strategies
def combine_vectors(keywords, custom_data_structures, weights, fusion):
    # Apply weights
    weighted_keywords = weights[0] * keywords
    weighted_custom_data_structures = weights[1] * custom_data_structures

    # Apply fusion
    if fusion == "add":
        max_length = max(weighted_keywords.shape[1], weighted_custom_data_structures.shape[1])
        weighted_keywords = pad_with_zeros(weighted_keywords, max_length)
        weighted_custom_data_structures = pad_with_zeros(weighted_custom_data_structures, max_length)
        combined_vectors = weighted_keywords + weighted_custom_data_structures
    elif fusion == "concatenate":
        combined_vectors = np.hstack((weighted_keywords, weighted_custom_data_structures))
    return combined_vectors

# Model definitions
catboost_params = {
    'verbose': 0,
    'iterations': 1000,
    'learning_rate': 0.05,
    'depth': 6,
    'l2_leaf_reg': 5,
    'early_stopping_rounds': 15,'random_state': 43
}

xgboost_params = {
    'n_estimators': 500,
    'learning_rate': 0.01,
    'max_depth': 5,
    'min_child_weight': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.8,
    'reg_alpha': 0.1,
    'reg_lambda': 1,'random_state': 43
}

random_forest_params = {
    'n_estimators': 100,
    'max_depth': 6,
    'min_samples_split': 5,
    'min_samples_leaf': 3,'random_state': 43
}

bagging_params = {
    'n_estimators': 100,
    'max_samples': 0.8,
    'max_features': 1.0,'random_state': 43
}

# Base models
models = {
    "CatBoost Regressor": CatBoostRegressor(**catboost_params),
    "XGBoost Regressor": XGBRegressor(**xgboost_params),
    "Random Forest Regressor": RandomForestRegressor(**random_forest_params),
    "Bagging Regressor": BaggingRegressor(**bagging_params)
}

# Create a stacking regressor using the models as base learners
estimators = [(name, model) for name, model in models.items()]
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=CatBoostRegressor(n_estimators=200, learning_rate=0.05, max_depth=3,verbose=0),
)

# Add the stacking model to the models dictionary
models["Stacking Regressor"] = stacking_model

# Load data without reduction
file_path = "/content/IG.xlsx"
keywords, custom_data_structures, final_marks = load_data(file_path)

# Weights and fusion types
weights_options = [(1, 1)]
fusions = ["concatenate"]

# Print results directly
for weights in weights_options:
    for fusion in fusions:
        combined_vectors = combine_vectors(keywords, custom_data_structures, weights, fusion)

        # Split data into train and test sets
        X_train, X_test, y_train, y_test = train_test_split(combined_vectors, final_marks, test_size=0.2, random_state=42, stratify=final_marks)

        print(f"\nFusion method: {fusion}")
        for name, model in models.items():
            model.fit(X_train, y_train)

            # Predict and calculate metrics for training set
            y_train_pred = np.round(model.predict(X_train))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            train_mse = mean_squared_error(y_train, y_train_pred)
            train_rmse = np.sqrt(train_mse)
            train_r2 = r2_score(y_train, y_train_pred)
            train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

            # Predict and calculate metrics for testing set
            y_test_pred = np.round(model.predict(X_test))
            test_mae = mean_absolute_error(y_test, y_test_pred)
            test_mse = mean_squared_error(y_test, y_test_pred)
            test_rmse = np.sqrt(test_mse)
            test_r2 = r2_score(y_test, y_test_pred)
            test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

            # Print results
            print(f"\nModel: {name}")
            print("Train Metrics:")
            print(f"MAE: {train_mae}, MSE: {train_mse}, RMSE: {train_rmse}, R2: {train_r2}, MAPE: {train_mape}")
            print("Test Metrics:")
            print(f"MAE: {test_mae}, MSE: {test_mse}, RMSE: {test_rmse}, R2: {test_r2}, MAPE: {test_mape}")



Fusion method: concatenate

Model: CatBoost Regressor
Train Metrics:
MAE: 0.5218274111675127, MSE: 0.7553299492385787, RMSE: 0.8690972035615916, R2: 0.8716082639960027, MAPE: 4572182362812.806
Test Metrics:
MAE: 1.311740890688259, MSE: 3.02834008097166, RMSE: 1.7402126539511371, R2: 0.4818055859403485, MAPE: 0.3326216181479339

Model: XGBoost Regressor
Train Metrics:
MAE: 0.9218274111675127, MSE: 1.5289340101522844, RMSE: 1.236500711747585, R2: 0.7401102781295776, MAPE: 18288729451250.977
Test Metrics:
MAE: 1.417004048582996, MSE: 3.3927125506072873, RMSE: 1.84193174428568, R2: 0.41945594549179077, MAPE: 0.3954646230962021

Model: Random Forest Regressor
Train Metrics:
MAE: 1.3380710659898478, MSE: 2.984771573604061, RMSE: 1.727649146558427, R2: 0.49264555933904286, MAPE: 22860911814063.78
Test Metrics:
MAE: 1.6153846153846154, MSE: 4.295546558704453, RMSE: 2.0725700371047666, R2: 0.2649675490410559, MAPE: 0.4649733307628044

Model: Bagging Regressor
Train Metrics:
MAE: 0.706598984771